In [1]:
import gym
import ray
from ray import tune
from ray.rllib.agents.ppo import PPOTrainer
from scouting_gym.tasks.scouting_discrete_task import ScoutingDiscreteTask


Instructions for updating:
non-resource variables are not supported in the long term
None


In [2]:
env = gym.make('Scouting-v0')

print(env.observation_space)

[ERROR] [1611745285.854981, 31.755000]: NOT Initialising Simulation Physics Parameters
[WARN] [1611745285.861055, 0.020000]: Start Init ControllersConnection
[WARN] [1611745285.862040, 0.020000]: END Init ControllersConnection


Box(0.0, 8.0, (12,), float32)


In [3]:
env.reset()
for _ in range(20):
    #obs, reward, done, _ = env.step(1)
    obs, reward, done, _ = env.step([0.6, ])
print(obs.min())
print(obs.max())

0.7753428816795349
7.99


In [4]:
print(obs)

[0.98392397 0.79180807 0.77534288 0.80873847 1.02237475 1.80956209
 1.86166608 2.06839633 2.9133904  2.31194305 7.99       7.99      ]


In [5]:
ray.init()

2021-01-27 12:01:29,727	INFO services.py:1171 -- View the Ray dashboard at http://127.0.0.1:8266


{'node_ip_address': '192.168.178.60',
 'raylet_ip_address': '192.168.178.60',
 'redis_address': '192.168.178.60:13851',
 'object_store_address': '/tmp/ray/session_2021-01-27_12-01-29_107005_3139395/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-01-27_12-01-29_107005_3139395/sockets/raylet',
 'webui_url': '127.0.0.1:8266',
 'session_dir': '/tmp/ray/session_2021-01-27_12-01-29_107005_3139395',
 'metrics_export_port': 59556,
 'node_id': 'dd7e1e5d6c0a02bd0911b0ca3c45e6bb18230633'}

In [6]:
config = {
    "env": ScoutingDiscreteTask,  # or "corridor" if registered above
    "env_config": {
        "corridor_length": 5,
    },
    # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.

    "num_gpus": 1,
    "num_workers": 1,  # parallelism
    "model": {
        "fcnet_hiddens": [256, ],
        #"fcnet_hiddens": tune.grid_search([[64, 64, ], [128, 128, ], [256, 256, ]])
    }
    #"model": {
    #    "dim": 40,
    #    "conv_filters": [[16, [3, 3], 2], [32, [3, 3], 2], [64, [3, 3], 2], [512, [5, 16], 1]]
    #}
    #"model": {
    #    "use_lstm": True,
    #    # Max seq len f
    #    or training the LSTM, defaults to 20.
    #    "max_seq_len": 20,
    #    # Size of the LSTM cell.
    #    "lstm_cell_size": 256
    #}
}

stop = {
    "episodes_total": 2500,
}

In [7]:
def train(stop_criteria, config, restorepath):
    """
    Train an RLlib PPO agent using tune until any of the configured stopping criteria is met.
    :param stop_criteria: Dict with stopping criteria.
        See https://docs.ray.io/en/latest/tune/api_docs/execution.html#tune-run
    :return: Return the path to the saved agent (checkpoint) and tune's ExperimentAnalysis object
        See https://docs.ray.io/en/latest/tune/api_docs/analysis.html#experimentanalysis-tune-experimentanalysis
    """
    analysis = ray.tune.run(PPOTrainer, config=config,
                            stop=stop_criteria,
                            checkpoint_freq=1,
                            checkpoint_at_end=True)
    # list of lists: one list per checkpoint; each checkpoint list contains 1st the path, 2nd the metric value
    checkpoints = analysis.get_trial_checkpoints_paths(trial=analysis.get_best_trial('episode_reward_mean', mode='max'),
                                                       metric='episode_reward_mean',
                                                       )
    # retriev the checkpoint path; we only have a single checkpoint, so take the first one
    checkpoint_path = checkpoints[0][0]
    return checkpoint_path, analysis

def load(checkpoint_path, config):
    """
    Load a trained RLlib agent from the specified path. Call this before testing a trained agent.
    :param path: Path pointing to the agent's saved checkpoint (only used for RLlib agents)
    """
    agent = PPOTrainer(config=config)
    agent.restore(checkpoint_path)
    return agent

def test(agent, env):
    """Test trained agent for a single episode. Return the episode reward"""
    # instantiate env class

    # run until episode ends
    episode_reward = 0
    done = False
    obs = env.reset()
    while not done:
        action = agent.compute_action(obs)
        obs, reward, done, info = env.step(action)
        episode_reward += reward

    return episode_reward

In [ ]:
checkpoint_path, analysis = train(stop_criteria=stop,
                                  config=config,
                                  restorepath='/home/dschori/ray_results/PPO_2021-01-26_21-34-32/PPO_ScoutingDiscreteTask_e5473_00000_0_2021-01-26_21-34-32/checkpoint_100/checkpoint-100')

Trial name,status,loc
PPO_ScoutingDiscreteTask_03325_00000,RUNNING,


(pid=3139643) WARNING:tensorflow:From /home/dschori/.local/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=3139643) Instructions for updating:
(pid=3139643) non-resource variables are not supported in the long term
(pid=3139643) 2021-01-27 12:01:34,884	INFO trainer.py:591 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
(pid=3139643) 2021-01-27 12:01:34,884	INFO trainer.py:616 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=3139639) WARNING:tensorflow:From /home/dschori/.local/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=3139639) Instructions for updating:
(pid=3139639) non-reso

(pid=3139643) None
(pid=3139639) None
Result for PPO_ScoutingDiscreteTask_03325_00000:
  custom_metrics: {}
  date: 2021-01-27_12-06-15
  done: false
  episode_len_mean: 41.02061855670103
  episode_reward_max: -93.22261968069297
  episode_reward_mean: -98.54206849810636
  episode_reward_min: -101.95727175831794
  episodes_this_iter: 97
  episodes_total: 97
  experiment_id: ae70aee8670745c387b63ebbd6aacd9d
  hostname: workstation
  info:
    learner:
      default_policy:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 4.999999873689376e-05
        entropy: 1.448979377746582
        entropy_coeff: 0.0
        kl: 0.004820636473596096
        model: {}
        policy_loss: -0.005949055776000023
        total_loss: 4568.89501953125
        vf_explained_var: 0.0038130630273371935
        vf_loss: 4568.90087890625
    num_steps_sampled: 4000
    num_steps_trained: 4000
  iterations_since_restore: 1
  node_ip: 192.168.178.60
  num_healthy_workers: 1
  off_policy_estimator: {}
  per

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_ScoutingDiscreteTask_03325_00000,RUNNING,192.168.178.60:3139643,1,271.352,4000,-98.5421,-93.2226,-101.957,41.0206


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_ScoutingDiscreteTask_03325_00000,RUNNING,192.168.178.60:3139643,2,520.093,8000,-98.683,-93.3086,-102.606,43.99


...

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_ScoutingDiscreteTask_03325_00000,RUNNING,192.168.178.60:3139643,12,2372.8,48000,-81.9418,117.774,-101.093,189.11


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_ScoutingDiscreteTask_03325_00000,RUNNING,192.168.178.60:3139643,13,2546.43,52000,-79.8363,117.774,-101.627,190.04


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_ScoutingDiscreteTask_03325_00000,RUNNING,192.168.178.60:3139643,14,2742.53,56000,-80.1064,117.774,-101.627,188.06


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_ScoutingDiscreteTask_03325_00000,RUNNING,192.168.178.60:3139643,15,3015.77,60000,-79.9574,117.774,-101.627,180.64


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_ScoutingDiscreteTask_03325_00000,RUNNING,192.168.178.60:3139643,16,3218.67,64000,-77.9071,118.143,-101.627,184.71


In [ ]:
checkpoint_path

In [ ]:
agent = load(checkpoint_path=checkpoint_path, config=config)

episode_reward = test(agent=agent, env=env)
episode_reward